# Combine HiC Sample Lane Fastqs
- **Author** - Frank Grenn
- **Date Started** - November 2019
- **Quick Description:** code to pull down sample files from google cloud and combine the lanes for each sample. So each sample will have two read fastqs in the end. runs on biowulf

In [ ]:
TEMPDIR_OUT="/path/to/temp_fastqs" #directory that will contain the combined lane fastqs
LANE_FASTQ_DIR=TEMPDIR_OUT+"/HICS" #directory we will copy all separate lane files to 

In [ ]:
import os
from os import listdir
from pathlib import Path
import pandas as pd


### pull down the files

In [ ]:
print(f"gsutil -m cp -r gs://foundin-raw-assay/ASSAYS/HICS {TEMPDIR_OUT}")

### list out the files and check how many we have

In [ ]:
filelist = listdir(LANE_FASTQ_DIR)

In [ ]:
print(len(filelist))
print(filelist[0:10])

### use some identifier in the sample file names to get unique names

In [ ]:
unique = [s for s in filelist if "L001_R1" in s]

In [ ]:
print(len(unique))
print(unique[0:10])

In [ ]:
sample_names = [x.replace("_L001_R1_001.fastq.gz","") for x in unique]

In [ ]:
print(len(sample_names))
print(sample_names[0:10])

In [ ]:
def format_combine_command(sample,read):
    return f"cat {LANE_FASTQ_DIR}/{sample}*_{read}_001.fastq.gz > {TEMPDIR_OUT}/{sample}/{sample}_{read}_001.fastq.gz".format(LANE_FASTQ_DIR,TEMPDIR_OUT)

### (optional) subset for testing

In [ ]:
sample_names = sample_names[0:2]
print(sample_names)

### make directories for all samples and then generate the swarm file to combine lanes

In [ ]:
for sample in sample_names:
    Path("{}/{}".format(TEMPDIR_OUT,sample)).mkdir(parents=True, exist_ok=True)

In [ ]:

out_file = open("{}/combine_samples.swarm".format(TEMPDIR_OUT),'w')
for sample in sample_names:
    print(sample)
    print(format_combine_command(sample,"R1") + '\n', file=out_file)
    print(format_combine_command(sample,"R2") + '\n' , file=out_file)
    


In [ ]:
!swarm -f {TEMPDIR_OUT}/combine_samples.swarm